Install & Import Packages

In [ ]:
!pip install supabase pandas python-dotenv pytz

In [3]:
import os
from datetime import datetime
import pytz # asia
import pandas as pd
from supabase import create_client
from dotenv import load_dotenv


Load Environment Variables and Create Supabase Client

This cell reads Supabase credentials from a .env file and connects to your database.


.env prevents exposing secrets in code.

In [4]:
# Load environment variables from .env file
load_dotenv()  # Make sure SUPABASE_URL and SUPABASE_KEY are in .env

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)


Helper Function for IST Date

In [5]:
def current_ist_date():
    """
    Get today's date in Asia/Kolkata timezone.
    This helps ensure consistent attendance records.
    """
    IST = pytz.timezone("Asia/Kolkata")
    return datetime.now(IST).strftime("%Y-%m-%d")

In [6]:
# Demonstration
print("Current IST Date:", current_ist_date())


Current IST Date: 2025-12-03


Create Classroom

This function creates a new classroom in Supabase.

Key features :

    Prevents duplicate class creation

    Adds default code and attendance limit

    Shows database insertion

In [ ]:
def create_classroom(class_name, code="1234", daily_limit=10):
    """
    Create a classroom in Supabase with a unique class name,
    a required code, and a daily attendance limit.
    """
    
    # Check if classroom already exists
    existing = (
        supabase.table("classroom_settings")
        .select("*")
        .eq("class_name", class_name)
        .execute()
        .data
    )
    
    if existing:
        print(f"⚠️ Class '{class_name}' already exists.")
        return
    
    # Insert new classroom settings
    supabase.table("classroom_settings").insert({
        "class_name": class_name,
        "code": code,
        "daily_limit": daily_limit,
        "is_open": True
    }).execute()
    
    print(f"✅ Classroom '{class_name}' created successfully.")


In [ ]:
# Demonstration
create_classroom("Demo_Class1", code="4321", daily_limit=5)

⚠️ Class 'Demo_Class1' already exists.


Add Students to Roll Map

This function maps roll numbers → student names.

Explaination:

    Avoids duplicate roll numbers

    Writes to roll_map table

In [11]:
def add_student(class_name, roll_number, name):
    """
    Add a student to a class by linking roll number to name.
    Prevents duplicate roll numbers.
    """
    
    # Check for duplicate roll number
    existing = (
        supabase.table("roll_map")
        .select("*")
        .eq("class_name", class_name)
        .eq("roll_number", roll_number)
        .execute()
        .data
    )
    
    if existing:
        print(f"⚠️ Roll {roll_number} already assigned to {existing[0]['name']}")
        return
    
    # Insert student into roll_map table
    supabase.table("roll_map").insert({
        "class_name": class_name,
        "roll_number": roll_number,
        "name": name
    }).execute()
    
    print(f"✅ Added {name} (Roll {roll_number}) to {class_name}")


In [12]:
# Demonstration
add_student("Demo_Class1", "1", "Ali")
add_student("Demo_Class1", "2", "Bob")
add_student("Demo_Class1", "3", "Chliey")


✅ Added Ali (Roll 1) to Demo_Class1
✅ Added Bob (Roll 2) to Demo_Class1
✅ Added Chliey (Roll 3) to Demo_Class1


In [13]:
add_student("Demo_Class1", "3", "Chl7ey")

⚠️ Roll 3 already assigned to Chliey


Mark Attendance

This is the most important function.

It checks:

    Correct code

    Roll number exists

    Attendance not already marked

    Daily limit not exceeded

In [14]:
def mark_attendance(class_name, roll_number, code):
    """
    Mark attendance for a student with multiple validations:
    - Correct class code
    - Student exists
    - Attendance not already marked today
    - Daily limit not exceeded
    """
    
    today = current_ist_date()
    
    # Fetch classroom settings
    settings = (
        supabase.table("classroom_settings")
        .select("*")
        .eq("class_name", class_name)
        .execute()
        .data[0]
    )
    
    # 1. Validate code
    if code != settings["code"]:
        print("❌ Incorrect code entered.")
        return
    
    # 2. Check if already marked
    existing = (
        supabase.table("attendance")
        .select("*")
        .eq("class_name", class_name)
        .eq("roll_number", roll_number)
        .eq("date", today)
        .execute()
        .data
    )
    
    if existing:
        print("⚠️ Attendance already marked today.")
        return
    
    # 3. Check daily limit
    count_today = (
        supabase.table("attendance")
        .select("*", count="exact")
        .eq("class_name", class_name)
        .eq("date", today)
        .execute()
        .count
    )
    
    if count_today >= settings["daily_limit"]:
        print("⚠️ Daily attendance limit reached.")
        return
    
    # 4. Validate student exists
    student = (
        supabase.table("roll_map")
        .select("*")
        .eq("class_name", class_name)
        .eq("roll_number", roll_number)
        .execute()
        .data
    )
    
    if not student:
        print("❌ Roll number not registered.")
        return
    
    name = student[0]["name"]
    
    # Insert attendance
    supabase.table("attendance").insert({
        "class_name": class_name,
        "roll_number": roll_number,
        "name": name,
        "date": today
    }).execute()
    
    print(f"✅ Attendance recorded for {name} ({roll_number})")


In [15]:
# Demonstration
mark_attendance("Demo_Class1", "1", "4321")
mark_attendance("Demo_Class1", "2", "4321")
mark_attendance("Demo_Class1", "3", "421")

✅ Attendance recorded for Ali (1)
✅ Attendance recorded for Bob (2)
❌ Incorrect code entered.


In [16]:
# Demonstration
mark_attendance("Demo_Class1", "1", "4321")
mark_attendance("Demo_Class1", "2", "4321")
mark_attendance("Demo_Class1", "3", "4321")

⚠️ Attendance already marked today.
⚠️ Attendance already marked today.
✅ Attendance recorded for Chliey (3)


Show Attendance Matrix

This converts attendance into a pivot table.

    Rows → students

    Columns → dates
    
    Values → P / A

basic analytics.

In [17]:
def attendance_matrix(class_name):
    """
    Create a pivot table showing attendance for every student across dates.
    """
    
    records = (
        supabase.table("attendance")
        .select("*")
        .eq("class_name", class_name)
        .order("date", desc=True)
        .execute()
        .data
    )
    
    if not records:
        print("No attendance found.")
        return
    
    df = pd.DataFrame(records)
    df["status"] = "P"  # Mark all fetched entries as present
    
    pivot_df = df.pivot_table(
        index=["roll_number", "name"],
        columns="date",
        values="status",
        aggfunc="first",
        fill_value="A"
    ).reset_index()
    
    return pivot_df


In [18]:
# Demonstration
attendance_matrix("Demo_Class1")

date,roll_number,name,2025-12-03
0,1,Ali,P
1,2,Bob,P
2,3,Chliey,P


Attendance Analytics

We compute:

    Total presents

    Attendance percentage

    Top 3

    Bottom 3

In [20]:
def attendance_analytics(class_name):
    records = supabase.table("attendance").select("*").eq("class_name", class_name).execute().data
    if not records:
        print("No attendance found")
        return

    df = pd.DataFrame(records)
    df["status"] = "P"
    pivot_df = df.pivot_table(index=["roll_number","name"], columns="date", values="status", aggfunc="first", fill_value="A").reset_index()
    
    date_cols = pivot_df.columns[2:]
    pivot_df["Present_Count"] = pivot_df[date_cols].apply(lambda row: sum(val=="P" for val in row), axis=1)
    pivot_df["Attendance %"] = (pivot_df["Present_Count"]/len(date_cols)*100).round(2)
    
    print("Top 3 students:")
    print(pivot_df.sort_values("Attendance %", ascending=False).head(3))
    
    print("\nBottom 3 students:")
    print(pivot_df.sort_values("Attendance %").head(3))


In [21]:
# Demonstration
attendance_analytics("Demo_Class1")

Top 3 students:
date roll_number    name 2025-12-03  Present_Count  Attendance %
0              1     Ali          P              1         100.0
1              2     Bob          P              1         100.0
2              3  Chliey          P              1         100.0

Bottom 3 students:
date roll_number    name 2025-12-03  Present_Count  Attendance %
0              1     Ali          P              1         100.0
1              2     Bob          P              1         100.0
2              3  Chliey          P              1         100.0


ALLL IN ONE

Run Experiment

In [22]:
# Create demo classroom
create_classroom("Demo_Class", code="4321", daily_limit=5)

# Add students
add_student("Demo_Class", "1", "Alice")
add_student("Demo_Class", "2", "Bob")
add_student("Demo_Class", "3", "Charlie")

# Mark attendance
mark_attendance("Demo_Class", "1", "4321")
mark_attendance("Demo_Class", "2", "4321")
mark_attendance("Demo_Class", "3", "4321")

# Show matrix
attendance_matrix("Demo_Class")

# Analytics
attendance_analytics("Demo_Class")


✅ Classroom 'Demo_Class' created successfully.
✅ Added Alice (Roll 1) to Demo_Class
✅ Added Bob (Roll 2) to Demo_Class
✅ Added Charlie (Roll 3) to Demo_Class
✅ Attendance recorded for Alice (1)
✅ Attendance recorded for Bob (2)
✅ Attendance recorded for Charlie (3)
Top 3 students:
date roll_number     name 2025-12-03  Present_Count  Attendance %
0              1    Alice          P              1         100.0
1              2      Bob          P              1         100.0
2              3  Charlie          P              1         100.0

Bottom 3 students:
date roll_number     name 2025-12-03  Present_Count  Attendance %
0              1    Alice          P              1         100.0
1              2      Bob          P              1         100.0
2              3  Charlie          P              1         100.0
